In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import warnings
from cgi import test
from pathlib import Path
import time
from unittest import result
from pytorch_lightning import Trainer
from ray import tune
import torch

from train import DataModule, Regressor

sn.set(rc={'figure.figsize':(11.7,8.27)})
%matplotlib inline
%load_ext autoreload
%autoreload 2

warnings.filterwarnings("ignore")

In [ ]:
import datetime
from functools import partial
from vacances_scolaires_france import SchoolHolidayDates
calendar = SchoolHolidayDates()

ALLOWED_HORIZONS = [0.5,1,2,4]

other_cols = ["tcc","t2m","ssrd","ff100","u100","v100"]
data_folder = "./data_challenge/data"
def percent_rows_na(df):
    return (len(df)-len(df.dropna(axis=0)))*100/len(df)


def assign_meteo_date_lancement(dt):    
    
    if 0<= dt.hour < 6:
         hour_date_lancement = 0
    elif 6< dt.hour <= 12:
         hour_date_lancement = 6
    elif 12< dt.hour <= 18:
         hour_date_lancement = 12
    else:
         hour_date_lancement = 18
    return dt.replace(hour=hour_date_lancement)
         
         
def add_holiday_and_weekends_indicators(df):
    # WARNING : use only for conso !
    df['is_weekend'] = (df.date_cible.dt.dayofweek > 4).astype(int).astype(float)
    df['date'] = df.date_cible.dt.date
    my_holiday_func = lambda zone, date : calendar.is_holiday_for_zone(date, zone)
    df['is_holiday_A'] = df.date.apply(partial(my_holiday_func,'A')).astype(int).astype(float)
    df['is_holiday_B'] = df.date.apply(partial(my_holiday_func,'B')).astype(int).astype(float)
    df['is_holiday_C'] = df.date.apply(partial(my_holiday_func,'C')).astype(int).astype(float)
    df.drop(columns='date',inplace=True)
    return df
    
def fix_echeance(df):
    df['echeance'] = (df.date_cible - df.date_lancement).dt.seconds/3600

def remove_useless_horizons(df):
    return df.loc[df.echeance.isin(ALLOWED_HORIZONS)]

def add_datetime_features(df):
    # time in the year
    #df['year_dt'] =  datetime.datetime(year=df.date_cible.dt.year)
    tzinfo = df.date_cible.dt.tz
    df['tiy'] = (df.date_cible - df.date_cible.dt.year.apply(lambda y: datetime.datetime(year=y,month=1,day=1,tzinfo=tzinfo))).dt.total_seconds()/(365*24*60*60)
    # time in the day
    df['tid'] = (df.date_cible.dt.hour *3600 + df.date_cible.dt.minute *60 + df.date_cible.dt.second)/(24*60*60)
    # TODO: type of day for consumption


In [ ]:
df_list_station = pd.read_csv(os.path.join(data_folder,"liste_stations.csv"), sep=";", header=0)
df_list_station.head()

In [ ]:
df_prev_sans_obs2020 = pd.read_feather(os.path.join(data_folder, "df_prev_sans_obs2020.feather"))
print(df_prev_sans_obs2020.echeance.unique()) # echeance 30min - 7h
print(df_prev_sans_obs2020.isnull().sum()) # Missing 417844 observations (for 2020)
# Add fake PI for conso in order to get a FC between 0 and 1
df_prev_sans_obs2020.loc[df_prev_sans_obs2020.type.str.contains('conso'),'pi'] = df_prev_sans_obs2020[df_prev_sans_obs2020.type.str.contains('conso')].obs.max() + 10**5
# Compute FCs
df_prev_sans_obs2020['obs_fc'] = df_prev_sans_obs2020['obs'] / df_prev_sans_obs2020['pi']
df_prev_sans_obs2020['prev_fc'] = df_prev_sans_obs2020['prev'] / df_prev_sans_obs2020['pi']
df_prev_sans_obs2020['error_fc'] = df_prev_sans_obs2020['obs_fc'] - df_prev_sans_obs2020['prev_fc']

add_datetime_features(df_prev_sans_obs2020)
df_prev_sans_obs2020



In [ ]:
df_grille_zoneclimat_fin18 = pd.read_feather(os.path.join(data_folder, "grille_zone_climatique_fin2018.feather"))
df_grille_zoneclimat_fin18.head(10)

In [ ]:
df_meteo_zone_eol = pd.read_feather(os.path.join(data_folder, "meteo_zone_echeance12_2016_2020_HRES_piEOL_smooth.feather"))
df_meteo_zone_eol.groupby('date_lancement_meteo').count()

In [ ]:
df_meteo_zone_eol = pd.read_feather(os.path.join(data_folder, "meteo_zone_echeance12_2016_2020_HRES_piEOL_smooth.feather"))
print(sorted(df_meteo_zone_eol.echeance.unique())) # echeance 0min - 11h30
assert df_meteo_zone_eol.isnull().sum().sum() == 0 # No missing value
# Long to large

df_meteo_zone_eol = df_meteo_zone_eol.pivot(index=["date_lancement_meteo","date_cible","echeance"], values=other_cols, columns="zone").reset_index()
assert df_meteo_zone_eol.isnull().sum().sum() == 0 # No missing value
df_meteo_zone_eol

In [ ]:
df_meteo_zone_pv = pd.read_feather(os.path.join(data_folder, "meteo_zone_echeance12_2016_2020_HRES_piPV_smooth.feather"))
print(f"echeances:{sorted(df_meteo_zone_pv.echeance.unique())}") # echeance 0min - 11h30
print(f"zones:{sorted(df_meteo_zone_pv.zone.unique())}") # echeance 0min - 11h30
assert df_meteo_zone_pv.isnull().sum().sum() == 0 # No missing value


# Long to large
other_cols = ["tcc","t2m","ssrd","ff100","u100","v100"]
df_meteo_zone_pv = df_meteo_zone_pv.pivot(index=["date_lancement_meteo","date_cible","echeance"], values=other_cols, columns="zone").reset_index()
assert df_meteo_zone_pv.isnull().sum().sum() == 0 # No missing value
df_meteo_zone_pv

In [ ]:
df_prodpv_fc_q90 = pd.read_feather(os.path.join(data_folder, "productionPV_FC_cielclair_q90.feather"))
df_prodpv_fc_q90.head()

# Preprocessing

In [ ]:

#df = remove_useless_horizons(df_prev_sans_obs2020)
df = df_prev_sans_obs2020

# Drop uselss horizons
df = df[df.echeance.isin([0.5,1,2,4])]

df['date_lancement_meteo'] = df.date_lancement.apply(assign_meteo_date_lancement)
df_pv = df[df.type =='photovoltaique'].drop(columns='type')
df_conso = df[df.type =='consommation'].drop(columns='type')
df_conso_res = df[df.type =='consommation_residuelle'].drop(columns='type')
df_eol = df[df.type =='eolien'].drop(columns='type')
# No missing data in year < 2020, prev
assert percent_rows_na(df_eol[df_eol.date_cible.dt.year<2020])==0.0 # No missing value in train
assert percent_rows_na(df_pv[df_pv.date_cible.dt.year<2020])==0.0 # No missing value in train
assert percent_rows_na(df_conso_res[df_conso_res.date_cible.dt.year<2020])==0.0 # No missing value in train
assert percent_rows_na(df_conso[df_conso.date_cible.dt.year<2020])==0.0 # No missing value in train

# Add holidays for conso
df_conso = add_holiday_and_weekends_indicators(df_conso) # SLOW
df_conso_res = add_holiday_and_weekends_indicators(df_conso_res) # SLOW



df_pv


## PV

In [ ]:
# PV
PV_USELESS_COLS = ['ff100','u100','v100']
df_pv_meteo = df_pv.merge(df_meteo_zone_pv.drop(columns=PV_USELESS_COLS+['echeance']), on=['date_cible','date_lancement_meteo'], how='inner').drop(columns='date_lancement_meteo')

df_pv_meteo

## EOL

In [ ]:
# LONG
EOL_USELESS_COLS = ['tcc','ssrd','t2m']
df_eol_meteo = df_eol.merge(df_meteo_zone_eol.drop(columns=EOL_USELESS_COLS+['echeance']), on=['date_cible','date_lancement_meteo'], how='inner').drop(columns='date_lancement_meteo')
# TODO check how many values are lost during inner join
df_eol_meteo

## CONSO

In [ ]:
CONSO_USELESS_COLS = ['ff100','u100','v100']
# WARNING: TODO USE REAL WEATHER DATA 


df_conso_meteo = df_conso.merge(df_meteo_zone_pv.drop(columns=PV_USELESS_COLS+['echeance']), on=['date_cible','date_lancement_meteo'], how='inner').drop(columns='date_lancement_meteo')
df_conso_meteo


df_conso_res_meteo = df_conso_res.merge(df_meteo_zone_pv.drop(columns=PV_USELESS_COLS+['echeance']), on=['date_cible','date_lancement_meteo'], how='inner').drop(columns='date_lancement_meteo')
df_conso_res_meteo


## Save features

In [ ]:
df_pv_meteo.to_hdf("./features/photovoltaique.hdf",key="data")
df_eol_meteo.to_hdf("./features/eolien.hdf",key="data")
df_conso_meteo.to_hdf("./features/consommation.hdf",key="data")
df_conso_res_meteo.to_hdf("./features/consommation_residuelle.hdf",key="data")

In [ ]:
# Proving that all input data have hourly resolution
px.line(df_conso_res_meteo[df_conso_res_meteo.echeance==1.0].sort_values('date_cible'),x='date_cible',y="prev")

## Training

In [ ]:
# Run train.py

## Inference

In [ ]:

from train import LABEL_NAME


OBS_TYPES = ['photovoltaique','eolien','consommation','consommation_residuelle']
SUBMISSION_COLS = ['date_cible','date_lancement','quantile_niveau','type','prev_q']
def prepare_submission(obs_type, results, df_type='test'):
    # TODO fix error here
    net=Regressor(results.best_config)

    # Predict quantiles - using dm.predict_loader would be cleaner but does not work....
    with results.best_checkpoint.as_directory() as loaded_checkpoint_dir:
        ckp = torch.load(Path(loaded_checkpoint_dir) / "checkpoint")
        net.load_state_dict(ckp['state_dict'])
    
    # Load data
    OG_df = pd.read_hdf(f'./features/{obs_type}.hdf')
    dm = DataModule(OG_df, label=LABEL_NAME, batch_size=results.best_config['batch_size'])
    dm.prepare_data()
    x = getattr(dm, "x_"+df_type)
    df = getattr(dm, "df_"+df_type)
    
    # Predict
    net.eval()
    outs = net(x).detach()
    
    quantiles_cols = [f"{level:.3f}" for level in  np.array(net.quantile_levels)]
    quantiles_df = pd.DataFrame(columns=quantiles_cols, data=outs)
    
    # Concat to original DF
    results_df = pd.concat([df, quantiles_df.set_index(df.index)],axis=1)
    
    # Remove useless echeances
    results_df = results_df[results_df.echeance.isin(ALLOWED_HORIZONS)]
    
    # Large to long
    results_df['id'] = results_df.index
    COLS_TO_KEEP = ['date_cible','date_lancement','pi','echeance','prev','obs']
    for col in quantiles_cols:
        results_df[col] += results_df['prev_fc']
    results_df = results_df[COLS_TO_KEEP+quantiles_cols]
    results_df =  pd.melt(results_df, id_vars=COLS_TO_KEEP,value_vars=quantiles_cols,var_name="quantile_niveau",value_name="prev_q")
    results_df['quantile_niveau'] = pd.to_numeric(results_df['quantile_niveau'])
    results_df['type'] = obs_type
    
    # Multiply by installed power 
    results_df['prev_q'] = results_df['prev_q'] * results_df['pi']
    results_df.drop(columns='pi', inplace=True)
    # Zeroing negative productions
    results_df.loc[results_df.prev_q < 0, 'prev_q'] = 0 
    return results_df
        



In [ ]:
BEST_EXP_BY_TYPE = dict(
    photovoltaique="./ray/photovoltaique/experiment_state-2022-09-07_17-34-41.json",
    eolien="./ray/eolien/experiment_state-2022-09-07_16-47-39.json",
    consommation="./ray/consommation/experiment_state-2022-09-07_17-05-00.json",
    consommation_residuelle="./ray/consommation_residuelle/experiment_state-2022-09-07_17-28-04.json"
)

outs = []
for obs_type in OBS_TYPES:
    # Best model so far
    run_to_beat = tune.ExperimentAnalysis(experiment_checkpoint_path=BEST_EXP_BY_TYPE[obs_type],default_metric="val/loss",default_mode="min")
    
    # Latest run 
    exp_path = max(Path(f"./ray/{obs_type}/").glob('*experiment*'), key=os.path.getctime) # Hopefully take latest
    results = tune.ExperimentAnalysis(experiment_checkpoint_path=exp_path,default_metric="val/loss",default_mode="min")
    print(f"""
          Preparing submission for {obs_type}...
          Using Experiment {exp_path}
          Best_loss so far: {run_to_beat.best_result['val/loss']}
          Validation loss: {results.best_result['val/loss']}
          Relative val loss increase (%; negative is good): {(results.best_result['val/loss'] - run_to_beat.best_result['val/loss'])*100/run_to_beat.best_result['val/loss']}
          """)
    

    outs.append(prepare_submission(obs_type, results))
submission = pd.concat(outs,axis=0)
submission.reset_index(inplace=True)
submission[SUBMISSION_COLS].to_feather(f'./submissions/AR_{time.time()}.feather', compression="zstd")


## Output analysis

In [ ]:
pd.options.plotting.backend = "plotly"
import plotly.express as px
import plotly.graph_objects as go


def plot_val_output(obs_type):
        # Plot val preds. NB: there are holes because of the split between train and val.
    exp_path = max(Path(f"./ray/{obs_type}/").glob('*experiment*'), key=os.path.getctime) # take latest
    results = tune.ExperimentAnalysis(experiment_checkpoint_path=exp_path,default_metric="val/loss",default_mode="min")
    print(f"""
            Preparing submission for {obs_type}...
            Using Experiment {exp_path}
            
            """)
    print(results.best_config)

    out = prepare_submission(obs_type, results, df_type='val')
    df_conso_sub = out[(out.quantile_niveau.isin([0.25, 0.75]))& (out.echeance==1.0)]
    df_conso_sub.sort_values(by='date_cible',inplace=True)

    fig = px.line(df_conso_sub, x='date_cible',y='prev_q',color='quantile_niveau', line_dash_sequence=['dash'])
    fig.add_trace(go.Scatter(x=df_conso_sub.date_cible,y=df_conso_sub.prev,name="prev",line=dict(width=4)))
    fig.add_trace(go.Scatter(x=df_conso_sub.date_cible,y=df_conso_sub.obs,name="obs",line=dict(width=4)))
    fig.update_traces(connectgaps=False) 
    return fig



In [ ]:
plot_val_output('consommation_residuelle')


In [ ]:
plot_val_output('consommation')

In [ ]:
plot_val_output('eolien')

In [ ]:
dm.df_val.info()

In [ ]:

def analyze_fiability(out):
    quantile_levels = np.arange(0.005,1.00,0.005)
    out['percentage'] = pd.NA
    # TODO: 1 select quantile
    # 2 compute % of distribution above it
    # 3 deduce loss
    # LOSS 
    out['percentage'] = out.apply(lambda row: (row.prev_q >= out.obs).sum()/len(out), axis=1)
    out['reliability'] = indicator(out['percentage'] - out['prev_q'])*10
    
    return out
analyze_fiability(out.head(1000))

In [ ]:
# Debug submission
obs_type = 'consommation'
df_type='train'
exp_path = max(Path(f"./ray/{obs_type}/").glob('*experiment*'), key=os.path.getctime) # take latest
results = tune.ExperimentAnalysis(experiment_checkpoint_path=exp_path,default_metric="val/loss",default_mode="min")
net=Regressor(results.best_config)

# Predict quantiles - using dm.predict_loader would be cleaner but does not work....
with results.best_checkpoint.as_directory() as loaded_checkpoint_dir:
    ckp = torch.load(Path(loaded_checkpoint_dir) / "checkpoint")
    net.load_state_dict(ckp['state_dict'])

# Load data
OG_df = pd.read_hdf(f'./features/{obs_type}.hdf')
dm = DataModule(OG_df, label='error_fc', batch_size=results.best_config['batch_size'])
dm.prepare_data()
x = getattr(dm, "x_"+df_type)
df = getattr(dm, "df_"+df_type)

px.line(df[df.echeance==1.0].sort_values('date_cible'),x='date_cible',y='prev')
# # Predict
# net.eval()
# outs = net(x).detach()

# print(f"""
#         Validation loss (centered): {results.best_result['val/loss']}
#         """)


# quantiles_cols = [f"{level:.3f}" for level in  np.array(net.quantile_levels)]
# quantiles_df = pd.DataFrame(columns=quantiles_cols, data=outs)

# # Concat to original DF
# results_df = pd.concat([df, quantiles_df.set_index(df.index)],axis=1)

# # Remove useless echeances
# results_df = results_df[results_df.echeance.isin(ALLOWED_HORIZONS)]

# # Large to long
# results_df['id'] = results_df.index
# COLS_TO_KEEP = ['date_cible','date_lancement','pi','echeance','prev','obs']
# for col in quantiles_cols:
#     results_df[col] += results_df['prev_fc']
# results_df = results_df[COLS_TO_KEEP+quantiles_cols]
# results_df =  pd.melt(results_df, id_vars=COLS_TO_KEEP,value_vars=quantiles_cols,var_name="quantile_niveau",value_name="prev_q")
# results_df['quantile_niveau'] = pd.to_numeric(results_df['quantile_niveau'])
# results_df['type'] = obs_type

# # Multiply by installed power 
# results_df['prev_q'] = results_df['prev_q'] * results_df['pi']
# results_df.drop(columns='pi', inplace=True)
# # Zeroing negative productions
# results_df.loc[results_df.prev_q < 0, 'prev_q'] = 0 


In [ ]:
df = pd.read_hdf('features/consommation.hdf')
dm = DataModule(df, LABEL_NAME,10)
dm.prepare_data()
# Show feature columns
print(dm.df_train.drop(columns=dm.cols_to_drop))
dm.df_train[LABEL_NAME].values

In [ ]:
# Check that df has all timesteps
dm.df_train.info()

## IDEAs
- coeff zone meteo ? altitude ? longitude ?
- is it damageable to scale features for quantile prediction ?
- train only on useful horizons ?
- add dvc 

## TODO
- Compute "real" score on best models (* PI) 
- add fiabilité

